# ASMSA: Rum molecular dynamics with CV derived AAE model

**Previous steps**
- [prepare.ipynb](prepare.ipynb): Download and sanity check input files
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training|
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule



In [ ]:
import asmsa
import os
import re
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# unlike previous notebooks, we use the initial PDB only

pdb = "trpcage_correct.pdb"
base = "trpcage"

In [ ]:
# Kubernetes magics, find out volume name

mnt=os.popen('mount | grep /home/jovyan').read()
pvcid=re.search('pvc-[0-9a-z-]+',mnt).group(0)
pvc=os.popen(f'kubectl get pvc | grep {pvcid} | cut -f1 -d" "').read().rstrip()
pvc

In [ ]:
# Python wrapper to Kubernetes calls to gromacs 

gmx = asmsa.GMX(pvc=pvc)

In [ ]:
# Create a Gromacs job in Kubernetes, and run the "pdb2gmx" command in it
# wait for job completition and delete it immediately (not the results :-))

gmx.start(f"pdb2gmx -f {pdb} -o {base}.gro -p {base}.top -water tip3p -ff amber94",wait=True,delete=True)

In [ ]:
# Dodecahedron box of suitable size (adjust eventually)

mdbox=2.0
gmx.start(f"editconf -f {base}.gro -o {base}-box.gro -c -d {mdbox} -bt dodecahedron",wait=True,delete=True)

In [ ]:
# Solvate

gmx.start(f"solvate -cp {base}-box.gro -cs spc216.gro -o {base}-solv.gro -p {base}.top",wait=True,delete=True)

In [ ]:
# Add ions

gmx.start(f"grompp -f ions.mdp -c {base}-solv.gro -p {base}.top -o ions.tpr",wait=True,delete=True)

In [ ]:
gmx.start(f"genion -s ions.tpr -o {base}-ions.gro -p {base}.top -pname NA -nname CL -neutral",input="13",wait=True,delete=True)

In [ ]:
# Energy minimization

gmx.start(f"grompp -f minim-sol.mdp -c {base}-ions.gro -p {base}.top -o em.tpr",wait=True,delete=True)

In [ ]:
# This job may run a bit longer, start it in background, with more cores and GPU

gmx.start(f"mdrun -v -deffnm em -pin on",cores=4,gpus=1)

In [ ]:
# Peek the last 15 lines of output, may be called repeatedly

gmx.log(tail=15)

In [ ]:
# Delete the job when completed

gmx.delete()

In [ ]:
# Isothermal-isochoric equilibration, following the same job pattern

gmx.start(f"grompp -f nvt.mdp -c em.gro -r em.gro -p {base}.top -o nvt.tpr",wait=True,delete=True)

In [ ]:
gmx.start(f"mdrun -pin on -deffnm nvt",cores=4,gpus=1)

In [ ]:
gmx.log(tail=15)

In [ ]:
gmx.delete()

In [ ]:
# And isothermal-isobaric equilibration

gmx.start(f"grompp -f npt.mdp -c nvt.gro -r nvt.gro -t nvt.cpt -p {base}.top -o npt.tpr",wait=True,delete=True)

In [ ]:
gmx.start(f"mdrun -pin on -deffnm npt",cores=4,gpus=1)

In [ ]:
gmx.log(tail=15)

In [ ]:
gmx.delete()

In [ ]:
# Extract temperature, pressure and density from the trajectories and plot them

gmx.start(f"energy -f npt.edr -o press.xvg",input=18,wait=True,delete=True)
gmx.start(f"energy -f npt.edr -o dens.xvg",input=24,wait=True,delete=True)
gmx.start(f"energy -f npt.edr -o temp.xvg",input=16,wait=True,delete=True)

In [ ]:
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

In [ ]:
# Now we are ready to run metadynamics with plumed.dat generated in train.ipynb
# Checkpoints are stored, hence the simulation can be restarted if it gets killed

!cp md.mdp.template md.mdp

# 200 ns
# mdsteps=100000000

# 10 ns
mdsteps=5000000

with open('md.mdp','a') as mdp:
    mdp.write("nsteps = %d\n" % mdsteps)

gmx.start(f"grompp -f md.mdp -c npt.gro -t npt.cpt -p {base}.top -o md.tpr",wait=True,delete=True)

In [ ]:
!touch HILLS
gmx.start(f"mdrun -pin on -deffnm md -plumed plumed.dat -cpi md.cpt -cpt 15",cores=4,gpus=1)

In [ ]:
gmx.log(tail=15)

In [ ]:
gmx.delete()